# MIDS W261 Machine Learning At Scale

Christopher Llop | christopher.llop@ischool.berkeley.edu <br>
Week 4 | Submission Date: 9/29/2015


<b>HW4.0.</b>
What is MrJob? How is it different to Hadoop MapReduce? 
What are the mapper_final(), combiner_final(), reducer_final() methods? When are they called?

<span style="color:green"><b>Answer:</b></span>
1. 

<b>HW4.1. </b>

What is serialization in the context of MrJob or Hadoop? 
When it used in these frameworks? 
What is the default serialization mode for input and outputs for MrJob? 

<span style="color:green"><b>Answer:</b></span>

<b>HW4.2</b>

Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001
V,1000,1          #Visit by Visitor 10001 to page id 1000
V,1001,1          #Visit by Visitor 10001 to page id 1001
V,1002,1          #Visit by Visitor 10001 to page id 1002
C,"10002",10002   #Visitor id 10001
V
Note: #denotes comments
to the format:

V,1000,1,C, 10001
V,1001,1,C, 10001
V,1002,1,C, 10001

Write the python code to accomplish this.

<b>HW 4.3</b> 

Find the 5 most frequently visited pages using mrjob from the output of 4.2 (i.e., transfromed log file).

<b>HW4.4</b>


Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.


<b>HW 4.5</b>

Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

In [1]:
# Number of unique products and product frequency can be solved together

In [2]:
%%writefile mapper.py
#!/usr/bin/python
import sys
item_inventory = {}

for line in sys.stdin:
    for item in line.rstrip('\n').split():
        item_inventory[item] = item_inventory.get(item, 0) + 1
            
for item, inventory in item_inventory.iteritems():
    print "{}\t{}".format(item, inventory)

Overwriting mapper.py


In [3]:
%%writefile reducer.py
#!/usr/bin/python
import sys

unique_item_count = 0
current_item_count = 0
current_item = ""

for line in sys.stdin:
    line = line.rstrip('\n').split()
    if current_item == line[0]:
        # If same item, add to count
        current_item_count += int(line[1])
    else:
        # If new item, print, increment unique, restart count
        if unique_item_count > 0:
            print current_item, current_item_count
        unique_item_count += 1
        current_item_count = int(line[1])
        current_item = line[0]
        
# Print final row of counts
print current_item, current_item_count

# Finally, print the number of unique items (will be on last row of reducer output)
print unique_item_count


Overwriting reducer.py


In [4]:
# Use chmod for permissions
!chmod a+x mapper.py
!chmod a+x reducer.py

In [5]:
# Move files and make directory
!hadoop fs -mkdir ./W261/In/HW3
!hdfs dfs -put ./ProductPurchaseData.txt ./W261/In/HW3/

15/09/22 16:31:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
mkdir: `W261/In/HW3': File exists
15/09/22 16:31:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `W261/In/HW3/ProductPurchaseData.txt': File exists


In [6]:
# HW3.1_a: Execute a job using Hadoop Streaming to generate 10,000 random integers and sort them.
def HW3_1a():
    !hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar \
    -Dmapreduce.job.maps=10 \
    -Dmapreduce.job.reduces=1 \
    -files ./mapper.py,./reducer.py \
    -mapper ./mapper.py  \
    -reducer ./reducer.py \
    -input ./W261/In/HW3/ProductPurchaseData.txt -output ./W261/Out/HW3_1_a    
    
HW3_1a()

15/09/22 16:32:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 16:32:06 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/22 16:32:06 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/22 16:32:06 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/22 16:32:07 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/22 16:32:08 INFO mapreduce.JobSubmitter: number of splits:1
15/09/22 16:32:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1814222973_0001
15/09/22 16:32:09 INFO mapred.LocalDistributedCacheManager: Localized file:/Users/cjllop/Code/MIDS/W261/HW/W3/mapper.py as file:/usr/local/Cellar/hadoop/hdfs/tmp/mapred/local/1442953928736/mapper.py
15/09/22 16:32:09 INFO mapred.LocalDistributedCacheManager: L

In [7]:
print "The first 5 results in the reducer output are:"
!hadoop fs -cat ./W261/Out/HW3_1_a/part-00000 | head -n5

print
print "The total number of unique items is:"
!hadoop fs -cat ./W261/Out/HW3_1_a/part-00000 | tail -n1


The first 5 results in the reducer output are:
15/09/22 16:32:32 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI11153 8	
DAI11223 155	
DAI11238 3	
DAI11257 1	
DAI11261 6	
cat: Unable to write to output stream.

The total number of unique items is:
15/09/22 16:32:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
12592	


In [8]:
# Largest basket and frequency of basket counts

In [9]:
%%writefile mapper.py
#!/usr/bin/python
import sys

basket_inventory = {}

for basket in sys.stdin:
#    for basket in line:
    basket = basket.rstrip('\n')
    basket_inventory[basket] = basket_inventory.get(basket, 0) + 1
            
# Note - this code assumes we can fit the ENTIRE document in memory. This isnt' best practice.
#   I should really updated this code to check memory and emit whenever memory hits a certain point.
for basket, inventory in basket_inventory.iteritems():
    print "{}\t{}".format(basket, inventory)


Overwriting mapper.py


In [10]:
%%writefile reducer.py
#!/usr/bin/python
import sys

unique_basket_count = 0
current_basket_count = 0
current_basket = ""
largest_basket = []
largest_basket_size = 0

for line in sys.stdin:
    line = line.rstrip('\n').split('\t')
    if current_basket == line[0]:
        # If same item, add to count
        current_basket_count += int(line[1])
    else:
        # If new item, print, increment unique, restart count
        if unique_basket_count > 0:
            print current_basket.rstrip('\n'), current_basket_count
        unique_basket_count += 1
        current_basket_count = int(line[1])
        current_basket = line[0]
        
    # Track the maximum basket size
    if len(current_basket.split()) > largest_basket_size:
        largest_basket_size = len(current_basket.split())
        largest_basket = [current_basket]
    elif len(current_basket.split()) == largest_basket_size:
        largest_basket.append(current_basket)
        
print current_basket.rstrip('\n'), current_basket_count
print "The largest basket(s) have {} items. There are {} such baskets: {}".format(
    largest_basket_size, len(largest_basket), largest_basket)

Overwriting reducer.py


In [12]:
# HW3.1_b: Execute a job using Hadoop Streaming to generate 10,000 random integers and sort them.
def HW3_1b():
    !hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar \
    -Dmapreduce.job.maps=10 \
    -Dmapreduce.job.reduces=1 \
    -files ./mapper.py,./reducer.py \
    -mapper ./mapper.py  \
    -reducer ./reducer.py \
    -input ./W261/In/HW3/ProductPurchaseData.txt -output ./W261/Out/HW3_1_b    
    
HW3_1b()


15/09/22 16:33:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 16:33:01 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/22 16:33:01 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/22 16:33:01 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/22 16:33:01 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/22 16:33:02 INFO mapreduce.JobSubmitter: number of splits:1
15/09/22 16:33:02 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1607662248_0001
15/09/22 16:33:02 INFO mapred.LocalDistributedCacheManager: Localized file:/Users/cjllop/Code/MIDS/W261/HW/W3/mapper.py as file:/usr/local/Cellar/hadoop/hdfs/tmp/mapred/local/1442953982366/mapper.py
15/09/22 16:33:02 INFO mapred.LocalDistributedCacheManager: L

In [13]:
print "The first 5 results in the reducer output are:"
!hadoop fs -cat ./W261/Out/HW3_1_b/part-00000 | head -n5

print
print "Printing largest baskets...."
!hadoop fs -cat ./W261/Out/HW3_1_b/part-00000 | tail -n1

The first 5 results in the reducer output are:
15/09/22 16:33:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI11223 ELE54102 SNA56249 SNA30755 FRO80039 SNA53220  1	
DAI11238 SNA82274 SNA96466 GRO88324 SNA43409 FRO35729 GRO83463 GRO30912 ELE34234 ELE26753 ELE45560 ELE99887 ELE23393 SNA31446 SNA40784 GRO71621  1	
DAI11290 DAI37288 ELE55848 ELE32164 DAI43747 GRO17794 DAI43223 ELE20196 SNA26019 ELE62598 SNA42528 DAI92600 DAI42083 GRO59710 FRO56832 ELE75000  1	
DAI11290 DAI55148 DAI62779 GRO17794 SNA70824 SNA32151 FRO75586  1	
DAI11555 ELE66810 GRO43642 ELE66600 FRO91992  1	
cat: Unable to write to output stream.

Printing largest baskets....
15/09/22 16:33:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
The largest basket(s) have 37 items. There are 2 such baskets: ['FRO31317 DAI94514 FRO49726 FRO83352 FRO61354 GRO35122

<span style="color:silver"><b>HW3.2.</b> (Computationally prohibitive but then again Hadoop can handle this)</span>

<span style="color:silver">Note: for this part the writeup will require a specific rule ordering but the program need not sort the output.</span>

<span style="color:silver">List the top 5 rules with corresponding confidence scores in decreasing order of confidence score 
for frequent (100>count) itemsets of size 2. </span>
<span style="color:silver">A rule is of the form: </span>

<span style="color:silver">(item1) ⇒ item2.</span>

<span style="color:silver">Fix the ordering of the rule lexicographically (left to right), 
and break ties in confidence (between rules, if any exist) 
by taking the first ones in lexicographically increasing order. 
Use Hadoop MapReduce to complete this part of the assignment; 
use a single mapper and single reducer; use a combiner if you think it will help and justify. </span>


<span style="color:green"><b>Answer:</b></span>
I will solve this using the Apriori algorithm. To do so, we must first tabulate the counts for each item. Then, when checking item pairs, we first will make sure that each item is 'frequent' before checking to see if the item pair is 'frequent'. If either of the two items in the proposed pair is not frequent, we can skip checking to see if the pair itself is frequent.

I will do this using inverted ordering. The mapper/combiner will output two sorts of records:

The first type of record will automatically sort to the top and will allow us to calculate single-item frequency before processing pairs:

\* ITEM_ID COUNT

The second type of record will count pairs. Because of the inverted ordering, we can be assured we can calculate the single-term frequency before processing the term pairs:

*ITEM_ID ITEM_ID COUNT


I will use the "pairs" approach, as explained in the code comments below.

In [14]:
%%writefile mapper.py
#!/usr/bin/python
import sys

# Formula is: I -> J = (I U J) / I

# Pairs Approach | Note, I'm traveling to see a sick family member this weekend. While
# stipes is faster and perhaps a better approach, I am making the design decision that, in 
# my case, time of the programmer is the resource that we need to take into account. You can 
# liken this to a business situation where a project cannot afford programing hours, but can
# have a longer runtime :). In future weeks, I'll make different decisions to experiment
# with stripes.

# To gain some efficiecny back, I'll write a quick combiner to help make things better.
# In this case, there is no way the combiner will hurt things. I have to write code similar 
# to the combiner for the reducer anyways, so I'll just let Hadoop decide if it has spare
# bandwidth to run the combiner. Remember: Hadoop doesn't guarantee it will run the combiner
# if it doesn't make sense to.

basket_inventory = {}
        
for basket in sys.stdin:
    # Get unique items in basket
    basket = list(set(basket.rstrip('\n').split()))

    # Output a single record for each item so that we can use order inversion in our
    # reduce side frequency count.
    for item1 in basket:
        for item2 in basket:
            if item1 == item2:
                print "* {}\t1".format(item1)
            else:
                print "{} {}\t1".format(item1, item2)



Overwriting mapper.py


In [15]:
%%writefile combiner.py
#!/usr/bin/python
import sys

#itempairs = ["* 55\t1","* 55\t1","* 66\t1","23 45\t1","23 45\t1","23 95\t1"]
currentpair = ""
currentcount = 0

for itempair in sys.stdin:
#for itempair in itempairs:
    itempair = itempair.rstrip('\n')
    # If multiple of one key in a row, sum
    if itempair.split('\t')[0] == currentpair:
        currentcount += int(itempair.split('\t')[1])
    else:
        # Otherwise, print and reset counters. Note - a combiner must print in the same format
        # as a mapper.
        if currentcount > 0:
            print "{}\t{}".format(currentpair, currentcount)
        currentpair = itempair.split('\t')[0]
        currentcount = int(itempair.split('\t')[1])
print "{}\t{}".format(currentpair, currentcount)


Overwriting combiner.py


In [16]:
%%writefile reducer.py
#!/usr/bin/python
import sys

currentpair = ""
currentcount = 0
itemlist_freq = {}
itempair_freq = {}
frequent_cutoff = 100

#itempairs = ["* 55\t1","* 55\t3","* 45\t51","* 66\t1","23 95\t1","55 45\t1","55 45\t1"]

#for itempair in itempairs:
for itempair in sys.stdin:
    itempair = itempair.rstrip('\n')
    itempair, count = itempair.split('\t')
    firstitem, seconditem = itempair.split(' ')
    
    # For all sorts of records, we are not assured that the combiner fully combined
    # things. As such, we make sure the reducer finishes combining.
    # If we have a repeated key, sum the count
    if itempair == currentpair:
        currentcount += int(count)
    else:
        # Otherwise, post process
        if currentcount > 0:
            # For "*" records, store into dictionary to use later
            if currentpair.split()[0] == "*":
                itemlist_freq[currentpair.split()[1]] = currentcount
#                    print "{}\t{}".format(currentpair, currentcount)
            elif firstitem != "*":
            # For pairs, first see if components pass the min value (100)
            # Note - this step isn't technically needed since we already have the
            # pair count, however, we implement it in the spirit of the Apriori Alg.
                if itemlist_freq.get(currentpair.split()[0], 0) > frequent_cutoff and itemlist_freq.get(
                    currentpair.split()[1], 0) > frequent_cutoff:
                    # If both components pass, check to see if the full count passes
                    if currentcount > frequent_cutoff:
                        # In this case, we can calculate confidence score and output
                        print "{} => {} = {}".format(currentpair.split()[0], currentpair.split(
                            )[1], float(currentcount)/itemlist_freq[currentpair.split()[0]])
        currentpair = itempair
        currentcount = int(count)

# Because of our loop structure, we'll need to check then output one last score
if itemlist_freq.get(currentpair.split()[0], 0) > frequent_cutoff and itemlist_freq.get(
    currentpair.split()[1], 0) > frequent_cutoff:
    # If both components pass, check to see if the full count passes
    if currentcount > frequent_cutoff:
        # In this case, we can calculate confidence score and output
        print "{} => {} = {}".format(currentpair.split()[0], currentpair.split(
            )[1], float(currentcount)/itemlist_freq[currentpair.split()[0]])


Overwriting reducer.py


In [19]:
# HW3.2: Run MapReduce with mapper, reducer, and combiner
def HW3_2():
    !hadoop jar /usr/local/Cellar/hadoop/2.6.0/libexec/share/hadoop/tools/lib/hadoop-streaming-2.6.0.jar \
    -Dmapreduce.job.maps=1 \
    -Dmapreduce.job.reduces=1 \
    -mapper ./mapper.py  \
    -combiner ./combiner.py \
    -reducer ./reducer.py \
    -input ./W261/In/HW3/ProductPurchaseData.txt -output ./W261/Out/HW3_2    
HW3_2()


15/09/22 16:34:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/22 16:34:12 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/22 16:34:12 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/22 16:34:12 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/22 16:34:12 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/22 16:34:12 INFO mapreduce.JobSubmitter: number of splits:1
15/09/22 16:34:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1521152290_0001
15/09/22 16:34:13 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
15/09/22 16:34:13 INFO mapred.LocalJobRunner: OutputCommitter set in config null
15/09/22 16:34:13 INFO mapreduce.Job: Running job: job_local1521152290_0001
15/09/22 16:34:1

In [20]:
print "The first 10 results in the reducer output are:"
!hadoop fs -cat ./W261/Out/HW3_2/part-00000 | head -n10

The first 10 results in the reducer output are:
15/09/22 16:35:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI16732 => FRO78087 = 0.566844919786	
DAI18527 => SNA44451 = 0.380597014925	
DAI22177 => DAI31081 = 0.0780577750461	
DAI22177 => DAI62779 = 0.234787953288	
DAI22177 => DAI63921 = 0.0835894283958	
DAI22177 => DAI75645 = 0.0755992624462	
DAI22177 => DAI83733 = 0.0774431468961	
DAI22177 => DAI85309 = 0.105716041795	
DAI22177 => ELE17451 = 0.124769514444	
DAI22177 => ELE26917 = 0.0823601720959	
cat: Unable to write to output stream.


<span style="color:green"><b>Answer:</b></span>
We were told we do not need to sort the top list in Hadoop. I will read them from file, after downloading the file from HDFS. The step below shows our final answer:

- ('DAI93865', 'FRO40251', '1.0')
- ('GRO85051', 'FRO40251', '0.999176276771')
- ('GRO38636', 'FRO40251', '0.990654205607')
- ('ELE12951', 'FRO40251', '0.990566037736')
- ('DAI88079', 'FRO40251', '0.986725663717')



In [21]:
results_list = []
with open("part-00000") as output:
    for line in output:
        results_list.append((line.strip('\n').split()[0], line.strip('\n').split()[2], line.strip('\n').split()[4]))

# Note - I confirmed there are not ties, so we do not need to worry about lex. order
for ranked_result in sorted(results_list,key=lambda x: x[2], reverse=True)[:5]:
    print ranked_result

('DAI93865', 'FRO40251', '1.0')
('GRO85051', 'FRO40251', '0.999176276771')
('GRO38636', 'FRO40251', '0.990654205607')
('ELE12951', 'FRO40251', '0.990566037736')
('DAI88079', 'FRO40251', '0.986725663717')


In [18]:
# This cell can be used to delete old output to allow re-run of any Hadoop script.
#!hadoop fs -rm -r ./W261/Out/HW3_1_a
#!hadoop fs -rm -r ./W261/Out/HW3_1_b
#!hadoop fs -rm -r ./W261/Out/HW3_2

<span style="color:silver"><b>HW3.3 [Note - We were instructed to skip this problem]</b></span>

<span style="color:silver">Benchmark your results using the pyFIM implementation of the Apriori algorithm
(Apriori - Association Rule Induction / Frequent Item Set Mining implemented by Christian Borgelt). </span>
<span style="color:silver">You can download pyFIM from here: </span>

http://www.borgelt.net/pyfim.html

<span style="color:silver">Comment on the results from both implementations (your Hadoop MapReduce of apriori versus pyFIM) 
in terms of results and execution times.</span>


<span style="color:silver"><b>HW3.4</b> (Conceptual Exercise)</span>

<span style="color:silver">Suppose that you wished to perform the Apriori algorithm once again,
though this time now with the goal of listing the top 5 rules with corresponding confidence scores 
in decreasing order of confidence score for itemsets of size 3 using Hadoop MapReduce.
A rule is now of the form: </span>

<span style="color:silver">(item1, item2) ⇒ item3 </span>

<span style="color:silver">Recall that the Apriori algorithm is iterative for increasing itemset size,
working off of the frequent itemsets of the previous size to explore 
ONLY the NECESSARY subset of a large combinatorial space. 
Describe how you might design a framework to perform this exercise.</span>

<span style="color:silver">In particular, focus on the following:</span>
  — <span style="color:silver">map-reduce steps required</span>
  - <span style="color:silver">enumeration of item sets and filtering for frequent candidates</span>


<span style="color:green"><b>Answer</b></span>:

If we were still restricted to a single mapper/reducer pair, we would continue in the manner set forth above. We would need to violate the Apriori algorithm by creating pairs (or stripes) for all possible 3-item sets, then in the reducer we would only process those 3-item sets comprised of passing 2-item sets. In that case, the data from the mapper would leverage an additional level of inverted ordering, outputting tuples as follows:

- (\*, \*, a)
- (\*, a, b)
- (a, b, c)

From this, can count all the a, b terms via the pre-sorted *, a, b

However, this thought exercise allows us to move beyond this simple limitation.

To run the Apriori algorithem using multiple Map-Reduce passes, I would first install a package such as MrJob which makes such iterative approaches more intuitive to develop. I would then, on the first pass, determine which tuples passed the threshold as was done in problem 3.2 above. The output from that reducer would be a single line file listing the tuples that passed the check as shown below:

- DAI93865 FRO40251 [tab] GRO85051 FRO40251  [tab] GRO38636 FRO40251 (etc.)

This file (which could also be stored as a list and passed to the next iteration) would be loaded into the mappers on the second iteration and saved as a dictionary. Because these list of tuples have been filtered using the threshold, it is reasonable to believe they would fit in memory (for extremely large datasets this assumption could be revisted). Then, when processing in the mapper, only pairs present in the dictionary would be output.

This process could be repeated as many times as needed to create larger itemsets via Apriori.


This concludes HW 3.0. Thanks for reading!